In [1]:
import sys

sys.path.insert(0, '..')

import os
import requests
import json
from tempfile import mkdtemp
from urllib.request import urlretrieve
import libsbml
import amici
import os
from biosimulator_processes.utils import prepare_single_ode_process_document
from biosimulator_processes.data_model.compare_data_model import ODEComparisonDocument, DocumentFactory
from biosimulator_processes import CORE
from process_bigraph import Composite, pp

CobraProcess registered successfully.
CopasiProcess registered successfully.
_CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.
AmiciProcess registered successfully.
ODEComparatorProcess registered successfully.
GetSbmlStep registered successfully.
CompositionPlotter registered successfully.
Plotter2d registered successfully.
ODEComparatorStep registered successfully.


In [2]:
process_name = 'copasi'
module_name = f'{process_name}_process'
import_statement = f'biosimulator_processes.processes.{module_name}'
module_paths = module_name.split('_')
class_name = module_paths[0].replace(module_name[0], module_name[0].upper())
class_name += module_paths[1].replace(module_paths[1][0], module_paths[1][0].upper())


class_name, module_paths[1][0].upper()

('CopasiProcess', 'P')

In [3]:
module = __import__(
    import_statement, fromlist=[class_name])
# Get the class from the module
bigraph_class = getattr(module, class_name)

In [4]:
process = bigraph_class(config={'model': {'model_source': 'BIOMD0000000630'}})

found a biomodel id


In [5]:
# Step 1: Define the input parameters of the api method

biomodel_id = 'BIOMD0000000630'
sbml_dirpath = '../biosimulator_processes/model_files/sbml'
model_fp = os.path.join(sbml_dirpath, f'{biomodel_id}_url.xml')
species_context = 'concentrations'
species_port_name = f'floating_species_{species_context}'
species_store = [f'floating_species_{species_context}_store']
duration = 30
n_steps = 42
simulators = ['copasi', 'tellurium']

In [6]:
# Step 2: Define a simple function that encapsulates that which is defined in the rest of the workflow

def generate_ode_verification(biomodel_id, dur) -> dict:
    compare = {
        'compare_ode': {
                '_type': 'step',
                  'address': 'local:compare_ode_step',
                  'config': {'biomodel_id': biomodel_id, 'duration': dur},
                  'inputs': {},
                  'outputs': {
                    'comparison_data': ['comparison_store']
                }
        },
        'verification_data': {
                 '_type': 'step',
                  'address': 'local:ram-emitter',
                  'config': {
                      'emit': {
                          'comparison_data': 'tree[any]'
                      }
                  },
                  'inputs': {
                      'comparison_data': ['comparison_store']
                  }
        }
    }
    
    
    wf = Composite(config={'state': compare}, core=CORE)
    wf.run(1)
    comparison_results = wf.gather_results()
    return comparison_results, wf

In [7]:
# Step 3: View the results. TODO: Add RMSE here.

verification_results, wf = generate_ode_verification(biomodel_id, duration)

pp(verification_results)

2024-05-15 16:11:45.862 - amici.sbml_import - INFO - Finished importing SBML                         (2.81E-02s)
2024-05-15 16:11:45.886 - amici.sbml_import - INFO - Finished processing SBML observables            (2.14E-02s)
2024-05-15 16:11:45.889 - amici.sbml_import - INFO - Finished processing SBML event observables      (8.34E-07s)
2024-05-15 16:11:45.911 - amici.de_model - INFO - Finished computing xdot                            (2.26E-03s)
2024-05-15 16:11:45.915 - amici.de_model - INFO - Finished computing x0                              (1.43E-03s)
2024-05-15 16:11:45.922 - amici.de_model - INFO - Finished computing w                               (5.44E-03s)


found a filepath


2024-05-15 16:11:46.316 - amici.de_export - INFO - Finished generating cpp code                      (3.90E-01s)
2024-05-15 16:11:56.027 - amici.de_export - INFO - Finished compiling cpp code                       (9.71E+00s)
/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/../biosimulator_processes/processes/copasi_process.py:205: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



{ ('verification_data',): [ { 'comparison_data': { ('emitter',): [ { 'amici_simple_floating_species_concentrations': { 'plasmin': 0.0,
                                                                                                                       'plasminogen': 0.0,
                                                                                                                       'single intact chain urokinase-type plasminogen activator': 0.0,
                                                                                                                       'two-chain urokinase-type plasminogen activator': 0.0,
                                                                                                                       'x': 0.0,
                                                                                                                       'x-plasmin': 0.0},
                                                                     'copasi_simple_floating_species_

In [8]:
verification_results

{('verification_data',): [{'comparison_data': {('emitter',): [{'copasi_simple_floating_species_concentrations': {'plasminogen': 0.0,
       'plasmin': 0.0,
       'single intact chain urokinase-type plasminogen activator': 0.0,
       'two-chain urokinase-type plasminogen activator': 0.0,
       'x': 0.0,
       'x-plasmin': 0.0},
      'amici_simple_floating_species_concentrations': {'plasminogen': 0.0,
       'plasmin': 0.0,
       'single intact chain urokinase-type plasminogen activator': 0.0,
       'two-chain urokinase-type plasminogen activator': 0.0,
       'x': 0.0,
       'x-plasmin': 0.0},
      'tellurium_simple_floating_species_concentrations': {'species_1': 0.0,
       'species_2': 0.0,
       'species_3': 0.0,
       'species_4': 0.0},
      'time': 0.0},
     {'copasi_simple_floating_species_concentrations': {'plasminogen': 1.1758171177387002e+16,
       'plasmin': 1096150505274.1506,
       'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,

In [9]:
"""TODO: Update the input/output schema of the ODEComparatorProcess to mimic the following:

{('emitter',): [{
    'copasi': [
        {
            'floating_species_concentrations': {
                'plasminogen': 0.0,
                'plasmin': 0.0,
                'single intact chain urokinase-type plasminogen activator': 0.0,
                'two-chain urokinase-type plasminogen activator': 0.0,
                'x': 0.0,
                'x-plasmin': 0.0},
           'time': 0.0
        },
        ...
    ],
    'amici': [
        {
            'floating_species_concentrations': {
                'plasminogen': 1.1758171177387002e+16,
                'plasmin': 1096150505274.1506,
                'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,
                'two-chain urokinase-type plasminogen activator': 80249.33829510311,
                'x': 0.0,
                'x-plasmin': 0.0},
           'time': 0.0},
        },
       ...
    ]


"""

"TODO: Update the input/output schema of the ODEComparatorProcess to mimic the following:\n\n{('emitter',): [{\n    'copasi': [\n        {\n            'floating_species_concentrations': {\n                'plasminogen': 0.0,\n                'plasmin': 0.0,\n                'single intact chain urokinase-type plasminogen activator': 0.0,\n                'two-chain urokinase-type plasminogen activator': 0.0,\n                'x': 0.0,\n                'x-plasmin': 0.0},\n           'time': 0.0\n        },\n        ...\n    ],\n    'amici': [\n        {\n            'floating_species_concentrations': {\n                'plasminogen': 1.1758171177387002e+16,\n                'plasmin': 1096150505274.1506,\n                'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,\n                'two-chain urokinase-type plasminogen activator': 80249.33829510311,\n                'x': 0.0,\n                'x-plasmin': 0.0},\n           'time': 0.0},\n        },\n 

In [10]:
wf

In [11]:
wf.state.keys()

dict_keys(['compare_ode', 'verification_data', 'global_time', 'comparison_store'])

In [12]:
emitter = wf.state['verification_data']['instance']

In [13]:
emitter.history

[{'comparison_data': {('emitter',): [{'copasi_simple_floating_species_concentrations': {'plasminogen': 0.0,
      'plasmin': 0.0,
      'single intact chain urokinase-type plasminogen activator': 0.0,
      'two-chain urokinase-type plasminogen activator': 0.0,
      'x': 0.0,
      'x-plasmin': 0.0},
     'amici_simple_floating_species_concentrations': {'plasminogen': 0.0,
      'plasmin': 0.0,
      'single intact chain urokinase-type plasminogen activator': 0.0,
      'two-chain urokinase-type plasminogen activator': 0.0,
      'x': 0.0,
      'x-plasmin': 0.0},
     'tellurium_simple_floating_species_concentrations': {'species_1': 0.0,
      'species_2': 0.0,
      'species_3': 0.0,
      'species_4': 0.0},
     'time': 0.0},
    {'copasi_simple_floating_species_concentrations': {'plasminogen': 1.1758171177387002e+16,
      'plasmin': 1096150505274.1506,
      'single intact chain urokinase-type plasminogen activator': 2955755808974603.0,
      'two-chain urokinase-type plasminogen

In [14]:
wf.state['global_time']

1.0

In [15]:
outputs_1 = {
      "interval_id": 1,
      "copasi_floating_species_concentrations": {
        "plasminogen": 11758171177387002,
        "plasmin": 1096150505274.1506,
        "single intact chain urokinase-type plasminogen activator": 2955755808974603,
        "two-chain urokinase-type plasminogen activator": 80249.33829510311,
        "x": 0,
        "x-plasmin": 0
      },
      "tellurium_floating_species_concentrations": {
        "species_1": 0.009841338446759926,
        "species_2": 3.50091366956004e-7,
        "species_3": 0.003069285532211755,
        "species_4": 1.9636710780672987e-15
      },
      "amici_floating_species_concentrations": {
        "plasminogen": 0.01,
        "plasmin": 0,
        "single intact chain urokinase-type plasminogen activator": 0.002,
        "two-chain urokinase-type plasminogen activator": 0,
        "x": 0,
        "x-plasmin": 0
      },
      "time": 3
}

In [16]:
from biosimulator_processes.steps.comparator_step import generate_ode_process_comparison_matrix

species_data_index = 0

results = generate_ode_process_comparison_matrix(outputs_copasi=list(outputs_1['copasi_floating_species_concentrations'].values())[species_data_index],
                                                 outputs_amici=list(outputs_1['amici_floating_species_concentrations'].values())[species_data_index],
                                                 outputs_tellurium=list(outputs_1['tellurium_floating_species_concentrations'].values())[species_data_index])

ImportError: cannot import name 'generate_ode_process_comparison_matrix' from 'biosimulator_processes.steps.comparator_step' (/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/../biosimulator_processes/steps/comparator_step.py)

In [ ]:
results

In [17]:
from biosimulator_processes.data_model import DescriptiveArray
import numpy as np
import pandas as pd


output_copasi = DescriptiveArray(np.random.random(100), description='copasi output', process_id='simple_copasi')
output_tellurium = DescriptiveArray(np.random.random(100), description='tell output', process_id='simple_tellurium')

output_copasi.metadata, output_tellurium.metadata

({'process_id': 'simple_copasi', 'description': 'copasi output'},
 {'process_id': 'simple_tellurium', 'description': 'tell output'})

In [18]:
raw_outputs_matrix = np.stack([output_copasi, output_tellurium])

raw_outputs_matrix

array([[0.56411594, 0.20490422, 0.33876841, 0.06620329, 0.59412928,
        0.14393602, 0.01315893, 0.22866136, 0.66880953, 0.51090825,
        0.95772582, 0.73721859, 0.71086714, 0.91407988, 0.2704139 ,
        0.65043039, 0.19814454, 0.29708548, 0.77719613, 0.43339096,
        0.59955127, 0.03156151, 0.5473166 , 0.88509997, 0.82462473,
        0.88639994, 0.13841548, 0.88314265, 0.81221324, 0.74883894,
        0.20335625, 0.30825464, 0.66787244, 0.10690421, 0.94484007,
        0.56977596, 0.37395325, 0.06683676, 0.5313983 , 0.31231052,
        0.85569992, 0.58194492, 0.74890175, 0.16663918, 0.18336514,
        0.76208599, 0.83536756, 0.65093381, 0.95276298, 0.9134248 ,
        0.74919074, 0.28676903, 0.00903379, 0.73076931, 0.32200348,
        0.48196584, 0.11375004, 0.2649847 , 0.03710665, 0.9078739 ,
        0.29229425, 0.2555865 , 0.3836289 , 0.73357502, 0.71777923,
        0.6112131 , 0.24971265, 0.60789032, 0.1511908 , 0.79209861,
        0.80197881, 0.64499265, 0.56468069, 0.08

In [19]:
raw_outputs_df = pd.DataFrame(
    raw_outputs_matrix, 
    columns=[f'param_{str(n)}' for n in range(raw_outputs_matrix.shape[1])],
    index=[output_copasi.metadata['process_id'], output_tellurium.metadata['process_id']])

In [20]:
raw_outputs_df

,param_0,param_1,param_2,param_3,param_4,param_5,param_6,param_7,param_8,param_9,...,param_90,param_91,param_92,param_93,param_94,param_95,param_96,param_97,param_98,param_99
simple_copasi,0.564116,0.204904,0.338768,0.066203,0.594129,0.143936,0.013159,0.228661,0.668810,0.510908,...,0.268049,0.783307,0.446339,0.113843,0.071013,0.217745,0.831579,0.713358,0.899567,0.127818
simple_tellurium,0.229971,0.120802,0.023423,0.644007,0.614937,0.720251,0.182027,0.922137,0.354042,0.864182,...,0.447647,0.476756,0.987619,0.146348,0.580424,0.855239,0.996545,0.444023,0.666393,0.334349


In [21]:
transposed_outputs_df = raw_outputs_df.transpose()

transposed_outputs_df

,simple_copasi,simple_tellurium
param_0,0.564116,0.229971
param_1,0.204904,0.120802
param_2,0.338768,0.023423
param_3,0.066203,0.644007
param_4,0.594129,0.614937
...,...,...
param_95,0.217745,0.855239
param_96,0.831579,0.996545
param_97,0.713358,0.444023
param_98,0.899567,0.666393


In [22]:
transposed_outputs_df.index

Index(['param_0', 'param_1', 'param_2', 'param_3', 'param_4', 'param_5',
       'param_6', 'param_7', 'param_8', 'param_9', 'param_10', 'param_11',
       'param_12', 'param_13', 'param_14', 'param_15', 'param_16', 'param_17',
       'param_18', 'param_19', 'param_20', 'param_21', 'param_22', 'param_23',
       'param_24', 'param_25', 'param_26', 'param_27', 'param_28', 'param_29',
       'param_30', 'param_31', 'param_32', 'param_33', 'param_34', 'param_35',
       'param_36', 'param_37', 'param_38', 'param_39', 'param_40', 'param_41',
       'param_42', 'param_43', 'param_44', 'param_45', 'param_46', 'param_47',
       'param_48', 'param_49', 'param_50', 'param_51', 'param_52', 'param_53',
       'param_54', 'param_55', 'param_56', 'param_57', 'param_58', 'param_59',
       'param_60', 'param_61', 'param_62', 'param_63', 'param_64', 'param_65',
       'param_66', 'param_67', 'param_68', 'param_69', 'param_70', 'param_71',
       'param_72', 'param_73', 'param_74', 'param_75', 'param_

In [23]:
def calc_comparison(a, b):
    return (a - b) ** 2

In [24]:
np.random.random(3) - np.random.random(3)

array([ 0.2389997 , -0.40923965, -0.32169392])

In [25]:
transposed_comparisons = dict(zip(transposed_outputs_df.index, 
                                  [calc_comparison(*output_vals) for output_vals in transposed_outputs_df.values]))

In [26]:
for outputs in raw_outputs_matrix:
    print(outputs)

[0.56411594 0.20490422 0.33876841 0.06620329 0.59412928 0.14393602
 0.01315893 0.22866136 0.66880953 0.51090825 0.95772582 0.73721859
 0.71086714 0.91407988 0.2704139  0.65043039 0.19814454 0.29708548
 0.77719613 0.43339096 0.59955127 0.03156151 0.5473166  0.88509997
 0.82462473 0.88639994 0.13841548 0.88314265 0.81221324 0.74883894
 0.20335625 0.30825464 0.66787244 0.10690421 0.94484007 0.56977596
 0.37395325 0.06683676 0.5313983  0.31231052 0.85569992 0.58194492
 0.74890175 0.16663918 0.18336514 0.76208599 0.83536756 0.65093381
 0.95276298 0.9134248  0.74919074 0.28676903 0.00903379 0.73076931
 0.32200348 0.48196584 0.11375004 0.2649847  0.03710665 0.9078739
 0.29229425 0.2555865  0.3836289  0.73357502 0.71777923 0.6112131
 0.24971265 0.60789032 0.1511908  0.79209861 0.80197881 0.64499265
 0.56468069 0.08577791 0.75660962 0.28706844 0.22795046 0.79948323
 0.24725469 0.65748518 0.45311015 0.87936176 0.21768444 0.74789576
 0.34546498 0.68939451 0.18895945 0.24516962 0.06490915 0.835252

In [27]:
transposed_comparisons

{'param_0': 0.11165302372619355,
 'param_1': 0.007073248399158295,
 'param_2': 0.0994429531443158,
 'param_3': 0.3338566890164598,
 'param_4': 0.0004329461167123091,
 'param_5': 0.33213886150491223,
 'param_6': 0.02851645384915712,
 'param_7': 0.4809087394107249,
 'param_8': 0.09907873222616277,
 'param_9': 0.12480201192565492,
 'param_10': 0.3049749425547925,
 'param_11': 0.34390400573298874,
 'param_12': 0.0698056030173172,
 'param_13': 0.006715415509948594,
 'param_14': 0.02083292879006074,
 'param_15': 0.02718384891434933,
 'param_16': 0.011716706959747775,
 'param_17': 0.10107118343510169,
 'param_18': 0.0028027975193989853,
 'param_19': 0.11753899823996376,
 'param_20': 0.10197738524774576,
 'param_21': 0.7195515788294741,
 'param_22': 0.0009604426684352722,
 'param_23': 0.004123851952094822,
 'param_24': 0.030710483580833024,
 'param_25': 0.006340553839071711,
 'param_26': 0.1476927671302718,
 'param_27': 0.4938888187437539,
 'param_28': 0.009347334159317449,
 'param_29': 0.2096

In [28]:
comparison_data = {
    ('copasi', 'tellurium'): transposed_comparisons
}

In [29]:
comparison_data

{('copasi', 'tellurium'): {'param_0': 0.11165302372619355,
  'param_1': 0.007073248399158295,
  'param_2': 0.0994429531443158,
  'param_3': 0.3338566890164598,
  'param_4': 0.0004329461167123091,
  'param_5': 0.33213886150491223,
  'param_6': 0.02851645384915712,
  'param_7': 0.4809087394107249,
  'param_8': 0.09907873222616277,
  'param_9': 0.12480201192565492,
  'param_10': 0.3049749425547925,
  'param_11': 0.34390400573298874,
  'param_12': 0.0698056030173172,
  'param_13': 0.006715415509948594,
  'param_14': 0.02083292879006074,
  'param_15': 0.02718384891434933,
  'param_16': 0.011716706959747775,
  'param_17': 0.10107118343510169,
  'param_18': 0.0028027975193989853,
  'param_19': 0.11753899823996376,
  'param_20': 0.10197738524774576,
  'param_21': 0.7195515788294741,
  'param_22': 0.0009604426684352722,
  'param_23': 0.004123851952094822,
  'param_24': 0.030710483580833024,
  'param_25': 0.006340553839071711,
  'param_26': 0.1476927671302718,
  'param_27': 0.4938888187437539,
 

In [30]:
import json 
from biosimulator_processes.data_model.compare_data_model import ODEComparisonResult


with open('/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_outputs/test_ode_comparison_object.json', 'r') as f:
    output_data_response = json.load(f)

data_config = output_data_response.copy()
data_config.pop('outputs')
data_config.pop('timestamp')
    
    
output_data = ODEComparisonResult(**data_config)

2024-05-15 16:12:08.854 - amici.sbml_import - INFO - Finished loading SBML                           (5.86E-03s)
2024-05-15 16:12:08.874 - amici.sbml_import - INFO - Finished importing SBML                         (1.80E-02s)
2024-05-15 16:12:08.880 - amici.sbml_import - INFO - Finished processing SBML observables            (5.06E-03s)
2024-05-15 16:12:08.882 - amici.sbml_import - INFO - Finished processing SBML event observables      (5.83E-07s)
2024-05-15 16:12:08.900 - amici.de_model - INFO - Finished computing xdot                            (1.50E-03s)
2024-05-15 16:12:08.903 - amici.de_model - INFO - Finished computing x0                              (9.39E-04s)
2024-05-15 16:12:08.909 - amici.de_model - INFO - Finished computing w                               (4.17E-03s)


found a filepath


2024-05-15 16:12:09.156 - amici.de_export - INFO - Finished generating cpp code                      (2.45E-01s)
2024-05-15 16:12:17.876 - amici.de_export - INFO - Finished compiling cpp code                       (8.72E+00s)
/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/demos/../biosimulator_processes/processes/copasi_process.py:205: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [31]:
def extract_unique_output_names(output_data: ODEComparisonResult) -> list[str]:   
    output_names = []
    for i, interval_output in enumerate(output_data.outputs):
        output_attributes = vars(interval_output)
        for output_attribute_key, output_value in output_attributes.items():
            if isinstance(output_value, dict):
                output_names.append(output_attribute_key)
    return list(set(output_names))
    

In [32]:
extract_unique_output_names(output_data)

['tellurium_floating_species_concentrations',
 'amici_floating_species_concentrations',
 'copasi_floating_species_concentrations']

In [33]:
from biosimulator_processes.steps.comparator_step import construct_process_interval_matrix

In [34]:
import numpy as np
import random
import pandas as pd


output_copasi = np.array([1.3, 2.4, 4.3])
output_tellurium = np.array([1.3, 2.4, 4.5])
output_amici = np.array([1.3, 2.4, 4.6])

simulators = ['copasi', 'tellurium', 'amici']

interval_data = dict(zip(simulators, [output_copasi, output_tellurium, output_amici]))

interval_data['time_id'] = random.randint(0, 400)

matrix = construct_process_interval_matrix(
    outputs_copasi=interval_data['copasi'],
    outputs_tellurium=interval_data['tellurium'],
    outputs_amici=interval_data['amici'],
    time_id=interval_data['time_id'],
    rtol=1e-05,
    atol=1e-08,
)


# matrix_df = pd.DataFrame(matrix, index=simulators, columns=simulators)
matrix

array([[1.  , 0.04, 0.09],
       [0.04, 1.  , 0.01],
       [0.09, 0.01, 1.  ]])

In [35]:
matrix_df = pd.DataFrame(matrix, columns=simulators, index=simulators)

matrix_df

,copasi,tellurium,amici
copasi,1.00,0.04,0.09
tellurium,0.04,1.00,0.01
amici,0.09,0.01,1.00


In [36]:
tolerance_range = np.linspace(0, 1, 1000)

all_close_data = []

for i, tolerance in enumerate(tolerance_range):
    tol_data = calc_comparison(output_copasi, output_tellurium)
    all_close_data.append(tol_data)

In [37]:
output_copasi, output_tellurium, output_amici

(array([1.3, 2.4, 4.3]), array([1.3, 2.4, 4.5]), array([1.3, 2.4, 4.6]))

In [38]:
y_data = all_close_data
x_data = list(range(len(all_close_data)))

len(x_data), len(y_data)

(1000, 1000)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(x_data, y_data)

plt.show()

In [ ]:
comparison_matrices = []

for i, output in enumerate(output_data.outputs):
    output_attributes = vars(output)
    for dataKey, dataVals in output_attributes.items():
        if isinstance(dataVals, dict):
            print(f'is dict: {dataKey}')
            interval_data_dict = dataVals 
            for k, v in interval_data_dict.items():
                pass

In [54]:
import numpy as np

x = np.array([0, 1, 2, 3, 4])
y = np.array([0, 1.01, 2.02, 3.03, 4.04])


score = lambda a, b: (a - b) ** 2
z = np.stack([x, y], axis=1)

scores = [score(*output) * 100 for output in z]

scores


[0.0,
 0.010000000000000018,
 0.04000000000000007,
 0.08999999999999883,
 0.16000000000000028]